<a href="https://colab.research.google.com/github/mmistroni/OpenBB_Experiments/blob/main/OpenBBAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### What we will attempt to  do here is to create an Agent that will call Openbb functions and big query
### We will use Smolagents first and then we will expand little by little

# Importing Libraries

In [11]:
!pip install litellm

# Important!!!
#
# <---- Set your 'OPENAI_API_KEY' as a secret over there with the "key" icon
#
#
import os
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key
from google.colab import auth
auth.authenticate_user()


In [12]:
# Install the OpenBB Platform with all available extensions.
# Messages indicating package version conflicts at the end of installation can be safely ignored.
!pip install openbb-charting
!pip install openbb-yfinance
!pip install openbb-finviz
!pip install openbb-fmp

# There is also a nightly distribution available, openbb-nightly

In [13]:
from openbb import obb
from google.colab import userdata
obb.equity.price.historical(symbol='AAPL',provider='yfinance')


OBBject[T]

id: 068c1cf0-17f2-731d-8000-1ad498b946d4
results: [{'date': datetime.date(2024, 9, 10), 'open': 218.9199981689453, 'high': 2...
provider: yfinance
warnings: None
chart: None
extra: {'metadata': {'arguments': {'provider_choices': {'provider': 'yfinance'}, 's...

In [14]:
!pip install smolagents -U
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, InferenceClientModel

In [15]:
#import requests
!pip install google-api-python-client

import pandas as pd
import numpy as np
from datetime import date, datetime
!mkdir -p data
from apiclient.http import MediaIoBaseDownload
!pip install fsspec
!pip install gcsfs
from pandas.tseries.offsets import BDay
import numpy as np
from datetime import datetime, date
from pandas.tseries.offsets import BDay
from google.colab import auth
auth.authenticate_user()


print('Authenticated')

Authenticated


In [19]:
from google.cloud import storage
from googleapiclient.discovery import build
from apiclient.http import MediaIoBaseDownload
from datetime import date


def read_from_bucket(filename, bucket_name, to_panda=True):
  gcs_service = build('storage', 'v1')
  # The name for the new bucket
  holder = 'data/{}'.format(filename.split('/')[-1])
  print('Reading {} from {}'.format(filename,bucket_name))
  with open(holder, 'wb') as f:
    request = gcs_service.objects().get_media(bucket=bucket_name,
                                              object=filename)
    media = MediaIoBaseDownload(f, request)

    done = False
    while not done:
      # _ is a placeholder for a progress object that we ignore.
      # (Our file is small, so we skip reporting progress.)
      _, done = media.next_chunk()

  print('Download complete')
  if to_panda:
    return pd.read_csv(holder)


from google.cloud import bigquery
from google.cloud import bigquery_storage
from google.cloud import bigquery
from google.cloud import bigquery_storage

# Construct a BigQuery client object.

def get_bigquery_as_dataframe(sql_stmnt):
  client = bigquery.Client(project='datascience-projects')
  bqstorageclient = bigquery_storage.BigQueryReadClient()
  query_job = client.query(sql_stmnt)
  return query_job.result().to_dataframe()


def get_senate_disclosures(start_date:str = '2025-01-01',
                                       end_date:str = '2025-03-31')-> dict:
  sql = f"""SELECT
          DISTINCT AS_OF_DATE,
          TICKER
        FROM
          `datascience-projects`.`gcp_shareloader`.`senate_disclosures`
        WHERE
          DISCLOSURE LIKE '%Purchase%'
          AND AS_OF_DATE BETWEEN DATE '{start_date}'
          AND DATE '{end_date}' /* Replace with your desired Date A and Date B */
        ORDER BY
          AS_OF_DATE;
        """



  bqclient = bigquery.Client(project='datascience-projects')
  dataframe = (
    bqclient.query(sql)
    .result()
    .to_dataframe()
  )

  dataframe['date'] = pd.to_datetime(dataframe['date'])

  # Format the timestamp to a clear date string
  dataframe['date'] = dataframe['date'].dt.strftime('%Y-%m-%d')

  # Convert the DataFrame to a JSON string
  # The 'orient' parameter is crucial for the desired JSON structure.
  # 'records' is a common and useful format for APIs.
  json_data = dataframe.to_json(orient='records')

  return json_data



# Compare vs industry. Check metrics against every other company  until we can find industry / sector metrics

get_senate_disclosures()



'[{"AS_OF_DATE":1735776000000,"TICKER":"Ticker:$SKIUSD"},{"AS_OF_DATE":1736208000000,"TICKER":"Ticker:AEP"},{"AS_OF_DATE":1736208000000,"TICKER":"Ticker:BMY"},{"AS_OF_DATE":1736208000000,"TICKER":"Ticker:DUK"},{"AS_OF_DATE":1736208000000,"TICKER":"Ticker:EFC"},{"AS_OF_DATE":1736208000000,"TICKER":"Ticker:JNJ"},{"AS_OF_DATE":1736208000000,"TICKER":"Ticker:PSA"},{"AS_OF_DATE":1736208000000,"TICKER":"Ticker:ABBV"},{"AS_OF_DATE":1736208000000,"TICKER":"Ticker:AMGN"},{"AS_OF_DATE":1736294400000,"TICKER":"Ticker:HURA"},{"AS_OF_DATE":1736380800000,"TICKER":"Ticker:"},{"AS_OF_DATE":1736380800000,"TICKER":"Ticker:HD"},{"AS_OF_DATE":1736380800000,"TICKER":"Ticker:KO"},{"AS_OF_DATE":1736380800000,"TICKER":"Ticker:IBM"},{"AS_OF_DATE":1736380800000,"TICKER":"Ticker:JNJ"},{"AS_OF_DATE":1736380800000,"TICKER":"Ticker:PPG"},{"AS_OF_DATE":1736467200000,"TICKER":"Ticker:$BTCUSD"},{"AS_OF_DATE":1736467200000,"TICKER":"Ticker:$SOLUSD"},{"AS_OF_DATE":1736467200000,"TICKER":"Ticker:$XRPUSD"},{"AS_OF_DATE":1

In [ ]:
# Attempting a big query

In [ ]:
def read_bigquery_table(table_name):
  import bigframes.pandas as bf
  bf.options.bigquery.location = "US" #this variable is set based on the dataset you chose to query
  bf.options.bigquery.project = "datascience-projects" #this variable is set based on the dataset you chose to query
  return bf.read_gbq(f"datascience-projects.gcp_shareloader.{table_name}") #this variable is set based on the dataset you chose to query

### Smolagent tools

In [ ]:
from smolagents import CodeAgent, tool
#from openbb import obb
from google.colab import userdata

@tool
def get_historical_prices_for_ticker(ticker:str,
                                     start_date:str,
                                     end_date:str) -> dict:
  """
    Return historical prices for a ticker.
    Args:
        ticker (str): a string representing a ticker
        start_date(str): a string representing a date in the format YYYY-MM-DD
        end_date(str): a string representing a date in the format YYYY-MM-DD
  """

  return obb.equity.price.historical(symbol='AAPL', start_date=start_date,
         end_date=end_date, provider='yfinance')

@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion (str): The type of occasion for the party. Allowed values are:
                        - "casual": Menu for casual party.
                        - "formal": Menu for formal party.
                        - "superhero": Menu for superhero party.
                        - "custom": Custom menu.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

#open bb




In [ ]:
agent = CodeAgent(tools=[suggest_menu, get_historical_prices_for_ticker], model=InferenceClientModel())

agent.run("Give me historical prices of AAPL between 1st of March 2025 and 1st of May 2025.")

In [ ]:
from litellm import completion
from typing import List, Dict


def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get response"""
    response = completion(
        model="openai/gpt-4o",
        messages=messages,
        max_tokens=1024
    )
    return response.choices[0].message.content


what_to_help_with = input("What do you need help with?")

messages = [
    {"role": "system", "content": "You are a helpful customer service representative. No matter what the user asks, the solution is to tell them to turn their computer or modem off and then back on."},
    {"role": "user", "content": what_to_help_with}
]

response = generate_response(messages)
print(response)